In [1]:
import re
import pandas as pd
import numpy as np
import joblib

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import gensim
import nltk

from nltk.stem.snowball import SnowballStemmer

from gensim.corpora import Dictionary, MmCorpus
from gensim.models import ldamodel

from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.decomposition import PCA

PATH = "D:/Petakabar/what/"
bencana = "bencana/"
ekonomi = "ekonomi/"
kecelakaan = "kecelakaan/"
kesehatan = "kesehatan/"
kriminalitas = "kriminalitas/"
olahraga = "olahraga/"

In [2]:
old = pd.read_csv('perbandingan/berita-24-5.csv')
# print(old)
new = pd.read_csv('perbandingan/berita-24-5.csv') # need changes
# print(new)

In [3]:
# old.columns
old.columns

Index(['ID', 'berita_title', 'berita_date', 'berita_qdate', 'berita_desc',
       'berita_source', 'created_at', 'berita_topik_id', 'qe_what', 'ner_when',
       'ner_who', 'ner_prov', 'ner_kab', 'ner_kec', 'ner_neg', 'sev_death',
       'sev_injury', 'sev_lost', 'sev_sink', 'sev_plane', 'sev_moneyUSD',
       'sev_moneyIDR', 'sev_persen', 'sev_inflasi', 'sev_internasional',
       'sev_nasional', 'sev_provkota', 'sev_lost_items', 'sev_lost_money',
       'sev_rape', 'sev_house', 'sev_person', 'class_classification',
       'berita_summary'],
      dtype='object')

# Bencana

In [3]:
# Bencana Section (Topic id = 1)
old_bencana = old[old['berita_topik_id'] == 1]
old_bencana = old_bencana[['berita_title', 'berita_date', 'berita_desc', 'berita_source']]
old_bencana.rename(columns = {'berita_title':'title', 'berita_date':'date', 'berita_desc':'desc', 'berita_source':'source'}, inplace=True)
print(old_bencana.shape)
old_bencana.head(1)

(2773, 4)


,title,date,desc,source
0,Bagaimana Proses Terjadinya Gempa Bumi?,"Kamis, 15 Des 2022 17:02 WIB","\- Saat gempa bumi terjadi, tanah dan apa saja...",www.kompas.com


## Preprocess

In [4]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# previous way

# nltk.download('punkt')
# nltk.download('stopwords')

excluded_words = ["tempat", "waktu", "hari"]
# for lda ----
excluded_words = []
# ------------
NLTK_StopWords = stopwords.words('indonesian')
# print(NLTK_StopWords)
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali",
                      "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
# txt_stopword = pd.read_csv(PATH + bencana + "stopwords.txt", names= ["stopwords"], header = None)
# NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
# for lda ----
txt_stopword = pd.read_csv(PATH + bencana + "stopwords_add_v1.txt", names= ["stopwords"], header = None)
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
txt_stopword = pd.read_csv(PATH + bencana + "stopwords_add_v2.txt", names= ["stopwords"], header = None)
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
txt_stopword = pd.read_csv(PATH + bencana + "stopwords_add_v3.txt", names= ["stopwords"], header = None)
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
# ------------

# convert stopword string to list & append additional stopword
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [10]:
def preprocessing(berita):
        s = str(berita)
        s = s.lower()
        s = s.replace('\n', ' ')
        s = s.replace('\r', ' ')
        # s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
        # for lda ----
        s = re.sub(r'[^a-zA-Z\s]', ' ', s)
        # ------------
        tokens = [token for token in s.split(" ") if token != ""]
        T = [t for t in tokens if (
            (t in excluded_words) or (t not in NLTK_StopWords))]
        return T

In [68]:
old_bencana = old_bencana.dropna(subset=['desc'])
old_bencana.sort_values('desc', inplace=True)
old_bencana.drop_duplicates(subset='desc', keep=False, inplace=True)
df_train, df_test = train_test_split(old_bencana,test_size=0.1)
print(len(df_train))

old_bencana.head()

,title,date,desc,source
9646,"Imbas Banjir di Semarang, KA Tujuan Jakarta An...","Sabtu, 31 Des 2022 23:39 WIB",\r\n \r\n\- Imbas banjir yang menggenangi h...,www.tribunnews.com
648,"5 Titik Banjir di Tangerang Selatan Pagi Ini, ...","Jumat, 02 Des 2022 09:04 WIB",\r\n\- Banjir masih menggenangi sejumlah dae...,www.kompas.com
47,"Si Jago Merah Menyala di Balikpapan, Dua Rumah...","Senin, 28 Nov 2022 13:41 WIB",Api mendadak muncul dari salah satu rumah war...,www.kompas.com
1784,Banjir-Tanah Longsor Berulang hingga Menggenan...,"Rabu, 07 Des 2022 11:50 WIB",Badan Penanggulangan Bencana Daerah (BPBD) Ku...,www.kompas.com
585,"Banjir Natuna, Pemprov Kepri Bantu Rp 300 Juta...","Sabtu, 17 Des 2022 09:15 WIB",Banjir bandang melanda Kabupatan Natuna dan P...,www.kompas.com


In [19]:
df_train.to_csv(PATH + bencana + "corpus/df_train_unprerocessed.csv" , index = None, header=True)
df_test.to_csv(PATH + bencana + "corpus/df_test_unprerocessed.csv" , index = None, header=True)

In [21]:
desc_text_train_unpreprocessed = []
desc_text_test_unpreprocessed = []

for x in range(0, df_train.shape[0]):
  desc_text_train_unpreprocessed.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test_unpreprocessed.append(df_test.iloc[x, -2])

In [22]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test_unpreprocessed, PATH + bencana + "corpus/model/desc_text_test_unpreprocessed.pkl")
joblib.dump(desc_text_train_unpreprocessed, PATH + bencana + "corpus/model/desc_text_train_unpreprocessed.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
old_bencana.to_csv(PATH + bencana + 'corpus/dataset/df_total_unpreprocessed.csv', index=False)
df_test.to_csv(PATH + bencana + 'corpus/dataset/df_test_unpreprocessed.csv', index=False)
df_train.to_csv(PATH + bencana + 'corpus/dataset/df_train_unpreprocessed.csv', index=False)

In [69]:
for x in range(0, old_bencana.shape[0]):
    text = preprocessing(old_bencana.iloc[x, -2])
    old_bencana.iloc[x, -2] = (' '.join(text))

for x in range(0, old_bencana.shape[0]):
    text = preprocessing(old_bencana.iloc[x, -4])
    old_bencana.iloc[x, -4] = (' '.join(text)) 
old_bencana.head()

In [25]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

In [27]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, PATH + bencana + "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, PATH + bencana + "corpus/model/desc_text_train_new.pkl") #INI

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
old_bencana.to_csv(PATH + bencana + 'corpus/dataset/df_total_new.csv', index=False) #INI
df_test.to_csv(PATH + bencana + 'corpus/dataset/df_test.csv', index=False)
df_train.to_csv(PATH + bencana + 'corpus/dataset/df_train.csv', index=False)

In [28]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(278, 9398)
tfidf_train_matrix
(2495, 29277)


In [29]:
joblib.dump(tfidf_vectorizer, PATH + bencana + "corpus/vectorizer/vectorizer_new.pkl") #INI
joblib.dump(tfidf_test_matrix, PATH + bencana + "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, PATH + bencana + "corpus/matrix/tfidf_train_new.pkl") #INI

['D:/Petakabar/what/bencana/corpus/matrix/tfidf_train_new.pkl']

## Pencarian

In [93]:
df_total = pd.read_csv(PATH + bencana + 'corpus/dataset/df_total_new.csv')
df_total = df_total[pd.notnull(df_total['desc'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load(PATH + bencana + 'corpus/model/desc_text_train_new.pkl')
document_text_test= joblib.load(PATH + bencana + 'corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2773 entries, 0 to 2772
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2773 non-null   object
 1   date    2773 non-null   object
 2   desc    2773 non-null   object
 3   source  2773 non-null   object
dtypes: object(4)
memory usage: 86.8+ KB
None
------------------------------------------------------------------------------------------
278
2495


In [94]:
kueri='gempa'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load(PATH + bencana + "corpus/matrix/tfidf_train_new.pkl" )
tfidf_vectorizer = joblib.load(PATH + bencana + "corpus/vectorizer/vectorizer_new.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])

['gempa']


Top 10 most similar documents in corpus:
No ID Dokumen  :  9
Tanggal        :  Minggu, 25 Des 2022 20:20 WIB
Isi berita     :  100 kapal pinisi kapal wisata jenis labuan bajo berlindung gugusan pulau menghindari gelombang melanda perairan labuan bajo pantauan lokasi minggu 25 12 2022 sore kapal kapal wisata menumpuk perairan sisi utara hotel meruorah labuan bajo kapal kapal wisata terlindung pulau depannya kepala kantor kesyahbandaran otoritas pelabuhan ksop kelas iii labuan bajo hasan sadili membenarkan kapal kapal wisata berlindung cuaca buruk melanda labuan bajo hasan sadili membenarkan kapal kapal wisata berlindung pulau cuaca buruk kapal kapal wisata diaz berlindung pulau sabtu 24 12 2022 hari cuaca buruk melanda labuan bajo hujan lebat disertai angin kencang gelombang perairan labuan bajo prakiraan badan meteorologi klimatologi geofisika bmkg besok cuaca buruk angkutan sungai danau penyeberangan asdp indonesia ferry cabang labuan bajo manggarai barat nusa tenggara tim

## YAKE TFIDF BERT

In [30]:
import gensim
# import pandas as pd
import csv
from yake import KeywordExtractor
import math
from operator import itemgetter
from nltk import tokenize
from nltk.tokenize import word_tokenize
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
embedder = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')
import scipy.sparse


In [31]:
df_total = pd.read_csv(PATH + bencana + 'corpus/dataset/df_total_new.csv')
df_total = df_total[pd.notnull(df_total['desc'])]
print(df_total.info())
print ('-'*90)
document_text= joblib.load(PATH + bencana + 'corpus/model/desc_text_train_new.pkl')
print(len(document_text))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2773 entries, 0 to 2772
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2773 non-null   object
 1   date    2773 non-null   object
 2   desc    2773 non-null   object
 3   source  2773 non-null   object
dtypes: object(4)
memory usage: 86.8+ KB
None
------------------------------------------------------------------------------------------
2495


In [32]:
df_test = pd.read_csv(PATH + bencana + 'corpus/dataset/df_test.csv')
df_test = df_test[pd.notnull(df_test['desc'])]
print(df_test.info())
print ('-'*90)
document_text_test= joblib.load(PATH + bencana + 'corpus/model/desc_text_test.pkl')
print(len(document_text_test))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   278 non-null    object
 1   date    278 non-null    object
 2   desc    278 non-null    object
 3   source  278 non-null    object
dtypes: object(4)
memory usage: 8.8+ KB
None
------------------------------------------------------------------------------------------
278


In [33]:
df_train = pd.read_csv(PATH + bencana + 'corpus/dataset/df_train.csv')
df_train = df_train[pd.notnull(df_train['desc'])]
print(df_train.info())
print ('-'*90)
document_text_train= joblib.load(PATH + bencana + 'corpus/model/desc_text_train_new.pkl')
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2495 entries, 0 to 2494
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2495 non-null   object
 1   date    2495 non-null   object
 2   desc    2495 non-null   object
 3   source  2495 non-null   object
dtypes: object(4)
memory usage: 78.1+ KB
None
------------------------------------------------------------------------------------------
2495


In [35]:
paths = PATH + bencana + 'idwiki_word2vec_300.model'

# maximal value dari cosine similarity dalah 1

def similar(kata, proseske):
    most_similars = id_w2v.wv.most_similar(kata)
    arr = []

    # proses masukin ke csv
    f = open(PATH + bencana + 'bow_what_hapus.csv', 'a', encoding='UTF8', newline='')

    # f1 = open(PATH + bencana + "less_or_equal_62.csv", "a", encoding='UTF8', newline='')
    # f2 = open(PATH + bencana + "more_62.csv", "a", encoding='UTF8', newline='')

    for x in range(len(most_similars)):
        words = most_similars[x][0]
        similarity = most_similars[x][1]
        # print(words)
        # print(similarity)
        result = [words, similarity]
        # if similarity > maxDistance:
        #     maxDistance = similarity
        # elif similarity < minDistance:
        #     minDistance = similarity

        # if round(similarity, 2) <= 0.62:
        #     f1writer = csv.writer(f1)
        #     f1writer.writerow(result)
        # else:
        #     f2writer = csv.writer(f2)
        #     f2writer.writerow(result)
        
        arr.append((words, similarity))

    writer = csv.writer(f)
    writer.writerow([str(proseske), kata, most_similars])

    f.close()
    # f1.close()
    # f2.close()

    return(arr)


def tree(arr, proseske):
    hasil = []
    for x in range(len(arr)):
        for y in range(10):
            words = arr[x][y][0]
            similarity = arr[x][y][1]
            # if round(similarity, 2) <= 0.62:
            tes = similar(words, proseske)
            hasil.append(tes)
    return(hasil)

id_w2v = gensim.models.word2vec.Word2Vec.load(paths)

f = open(PATH + bencana + 'bow_what_hapus.csv', 'w', encoding='UTF8')
writer = csv.writer(f)
writer.writerow(['tingkat setelah parent', 'parent', 'similarity'])

# f1 = open(PATH + bencana + "less_or_equal_62.csv", "w", encoding='UTF8', newline='')
# writer = csv.writer(f1)
# writer.writerow(['Word', 'Similarity'])
# f1.close()

# f2 = open(PATH + bencana + "more_62.csv", "w", encoding='UTF8', newline='')
# writer = csv.writer(f2)
# writer.writerow(['Word', 'Similarity'])
# f2.close()

f.close()
x = 0

tes = similar('bencana', x)
data = []
data.append(tes)

while(True):
    if(x <= 3):
        x = x+1
        coba = tree(data, x)
        data = coba
        continue
    else:
        # print('maxDistance ' + str(maxDistance) + '\n')
        # print('minDistance ' + str(minDistance) + '\n')
        break

## Main Course

In [56]:
# Load bow dataset

#jika dijalankan ulang, jangan lupa untuk men-deduplikasi, menghapus kosakata selain indonesia, dan Rencana Induk Perencanaan Bencana
df_bow_what = pd.read_csv(PATH + bencana + "bow/bow_what_old.csv")
df_bow_what.head()

,tingkat setelah parent,parent,similarity
0,2.0,banjir,"[('longsor', 0.7020419836044312), ('kekeringan..."
1,2.0,kelaparan,"[('wabah', 0.7254995703697205), ('epidemi', 0...."
2,2.0,gempa,"[('tsunami', 0.6986302137374878), ('letusan', ..."
3,2.0,kekeringan,"[('kelaparan', 0.6980215907096863), ('erosi', ..."
4,2.0,kebakaran,"[('ledakan', 0.6575784087181091), ('pemadaman'..."


In [57]:
# Ambil parent dari bow
bow_list_what = []

for i in range(0, df_bow_what.shape[0]):
  bow_list_what.append(df_bow_what.iloc[i, 1])

print("What ", bow_list_what[0])

print({
  'What': len(bow_list_what),
})

What  banjir
{'What': 268}


In [38]:
# Cari dok pertama Use data train
def cari_dokpertama(kueriAsli: str):
    kueriPre = preprocessing(kueriAsli)
    kueriPre = " ".join(kueriPre)
    hasilSearch = []
    
    tfidf_matrix = joblib.load(PATH + bencana + 'corpus/matrix/tfidf_train_new.pkl')
    tfidf_vectorizer = joblib.load(PATH + bencana + 'corpus/vectorizer/vectorizer_new.pkl')

    query_vec = tfidf_vectorizer.transform([kueriPre])
    results = cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
    for i in results.argsort()[-5:][::-1]:
        hasilSearch.append(df_total.iloc[i,-2])
    hasilSearch=". ".join(hasilSearch)
    
    return hasilSearch

In [39]:
##Keywords Extraction with YAKE
def keyword_yake(hasilSearch: str):
    keywordYake=[]

    k_extractor = KeywordExtractor(lan="id", n=1, top=50)
    k_extractor2 = KeywordExtractor(lan="id", n=2, top=50)
    keywords = k_extractor.extract_keywords(text=hasilSearch)
    # keywords = k_extractor2.extract_keywords(text=hasilSearch)
    keywords.extend(k_extractor2.extract_keywords(text=hasilSearch))
    keywordYake = [x for x, y in keywords]
    # keywordYake.append(keywords)
    print('*'*120)
    print('*'*120)
    print("Keyword yake")
    print(keywordYake)
    print('*'*120)
    print('*'*120)
    
    return keywordYake

In [40]:
#Keywords Extraction with TFIDF
def keyword_tfidf(hasilSearch: str):

    keywordtfidf=[]
    keywordtfidf2=[]

    total_words = re.sub(r'[^\w]', ' ', hasilSearch)
    total_words = total_words.lower().split()
    #print (total_words)
    total_word_length = len(total_words)
    total_sentences = tokenize.sent_tokenize(hasilSearch)
    total_sent_len = len(total_sentences)

    tf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if (each_word in excluded_words) or (each_word not in NLTK_StopWords):
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))

    idf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if (each_word in excluded_words) or (each_word not in NLTK_StopWords):
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

    #print(idf_score)
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    def get_top_n(dict_elem, n):
        result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
        hasil =list(result.keys())
        #print(list(result.keys()))        
        return hasil
    #print(get_top_n(tf_idf_score, 25))
    #print(len(get_top_n(tf_idf_score, 1)))
    keywordtfidf.append(get_top_n(tf_idf_score, 25))
    for i in range(len(keywordtfidf)):
        #print (i)
        totalKw=0
        totalKw=len(keywordtfidf[i])
        for j in range(totalKw):
            #print (j)
            keywordtfidf2.append(keywordtfidf[i][j])
    
    print('*'*120)
    print('*'*120)
    print("Keyword TFIDF")
    print (keywordtfidf2)
    print('*'*120)
    print('*'*120)

    return keywordtfidf2

In [41]:
#Keywords Extraction with BERT
def keyword_bert(hasilSearch: str):

    keywordbert=[]

    #for j in range(len(array_text)):
    keyword1 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 1))
    keyword2 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 2))

    #print("Keywords of article\n", keywords)
    for i in range (0,len (keyword1)):
        keywordbert.append(keyword1[i][0])
        keywordbert.append(keyword2[i][0])
    
    print('*'*120)
    print('*'*120)
    print("Keyword Bert")
    print (keywordbert)
    print('*'*120)
    print('*'*120)
    
    return keywordbert

In [42]:
# Borda rangking
def rangking (keywordGabung, kueriAsli: str):
    kandidatFinalCek=[]
    kandidatFinalFix=[]
    
    for i in keywordGabung:
        if (i not in kandidatFinalCek and i!=0):
            kandidatFinalCek.append(i)
    queries=[kueriAsli]
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(kandidatFinalCek)
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 80
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFinalFix.append(kandidatFinalCek[idx])

    print('*'*120)
    print('*'*120)
    print ('Kandidat Final Fix Rank: ', kandidatFinalFix)
    print('*'*120)
    print('*'*120)

    return kandidatFinalFix

In [43]:
# Keyword bow
def keywordCustomBow(bowList, initialQuery: str):
    cekDuplicate = []
    kandidatFix = []

    for i in bowList:
        if(i not in cekDuplicate and i!=0):
            cekDuplicate.append(i)

    queries=[initialQuery]
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(cekDuplicate)
    
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 500
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFix.append(cekDuplicate[idx])
    
    print('*'*120)
    print('*'*120)
    print("Keyword BoW")
    print ('Kandidat BoW: ', kandidatFix)
    print('*'*120)
    print('*'*120)

    return kandidatFix

In [58]:
# Creating query for what, when, where, who
what_initial_query = "bencana apa yang terjadi dalam berita"

what_query = preprocessing(what_initial_query)
what_query = " ".join(what_query)
print (what_query)

bencana berita


In [45]:
# Prepare w data
def prepareWData(initial_query: str, bow_list):
    hasilkandidat = []
    keywordGabung = []
    qeGabungan = []
    kueriFix = []

    hasilSearch     = cari_dokpertama(initial_query)
    # (ini yake + tfidf + bert) = qe statistik
    keywordYake     = keyword_yake(hasilSearch) # 20
    keywordtfidf2   = keyword_tfidf(hasilSearch) # 20
    keywordbert     = keyword_bert(hasilSearch) # 20
    # ini qe bow
    keywordBoW      = keywordCustomBow(bow_list, initial_query)

    for keyword1 in keywordYake:
        keywordGabung.append(keyword1)
    for keyword2 in keywordtfidf2:
        keywordGabung.append(keyword2)
    for keyword3 in keywordbert:
        keywordGabung.append(keyword3)  

    # hasilrank = qe statistik
    hasilrank = rangking(keywordGabung, initial_query)
    
    for word1 in hasilrank:
        kueriFix.append(word1)

    for word2 in keywordBoW:
        kueriFix.append(word2)

    for word3 in kueriFix:
        hasilkandidat.append(word3)

    kueriFix = [preprocessing(i) for i in kueriFix]
    
    qeGabunganDelimiter = []

    for word4 in kueriFix:
        for subWord in word4:
            qeGabungan.append(subWord)
            qeGabunganDelimiter.append(subWord)

    qeGabunganDelimiter = list(dict.fromkeys(qeGabunganDelimiter))
    qeGabungan = list(dict.fromkeys(qeGabungan))

    # (hasil ranking + bow) = kandidat final
    qeGabungan = [" ".join(qeGabungan)]

    print('*'*120)
    qeStatistik = hasilrank
    qeBoW = keywordBoW

    return [qeGabungan, qeStatistik, qeBoW, qeGabunganDelimiter]

In [59]:
whatResultList = prepareWData(what_query, bow_list_what)
print("What")
print()
qeGabunganWhat = whatResultList[0]
qeStatistikWhat = whatResultList[1]
qeBoWWhat = whatResultList[2]
qeGabunganDelimiterWhat = whatResultList[3]

************************************************************************************************************************
************************************************************************************************************************
Keyword yake
['sungai', 'mencapai', 'kebakaran', 'hutan', 'ciujung', 'air', 'lebak', 'febby', 'siaga', 'longsor', 'tma', 'hujan', 'warga', 'deras', 'debit', 'hari', 'rumah', 'rusak', 'edy', 'bencana', 'sentimeter', 'ciberang', 'sumut', 'tanah', 'duddy', 'pupr', 'jembatan', 'rahmayadi', 'berstatus', 'kepala', 'status', 'kecamatan', 'jalan', 'tercatat', 'cuaca', 'aman', 'relawan', 'daerah', 'upt', 'dinas', 'pembangunan', 'kehutanan', 'dampri', 'kabupaten', 'video', 'lokasi', 'bantaran', 'wilayah', 'berat', 'april', 'ciberang ciujung', 'tma mencapai', 'kebakaran hutan', 'sungai ciujung', 'ciujung lebak', 'sungai', 'sungai ciberang', 'lebak febby', 'berstatus siaga', 'tercatat tma', 'sentimeter aman', 'aman debit', 'debit air', 'febby cuaca', 'ciujung 

In [60]:
print(qeGabunganDelimiterWhat)
print(qeGabunganWhat)
print(qeBoWWhat)
print(qeStatistikWhat)

['bencana', 'kebencanaan', 'penanggulangan', 'febby', 'cuaca', 'hulu', 'ekstrem', 'rusak', 'meluap', 'kehutanan', 'kedungsumber', 'ciberang', 'dampri', 'menghancurkan', 'kesiapsiagaan', 'mengguyur', 'siaga', 'kebakaran', 'sindangkerta', 'pengecekan', 'kesatuan', 'terpantau', 'pembangunan', 'keterangan', 'gununghalu', 'langganan', 'pratama', 'menimbun', 'deras', 'ciujung', 'wilayah', 'mengakibatkan', 'bantaran', 'pemukiman', 'tma', 'pepohonan', 'jati', 'temayang', 'lingkungan', 'manusia', 'hutan', 'mengimbau', 'lebak', 'kabupaten', 'pengendalian', 'daerah', 'diguyur', 'banten', 'kecamatan', 'kemiringannya', 'hujan', 'hari', 'status', 'tanah', 'upt', 'menyebabkan', 'menyiagakan', 'wib', 'sambungnya', 'warga', 'debit', 'airnya', 'pencegahan', 'mengkhawatirkan', 'jalan', 'menetapkan', 'dinas', 'menargetkan', 'pengelolaan', 'air', 'tercatat', 'sekitaran', 'perkecambahan', 'letusan', 'benturan', 'gesekan', 'badai', 'gempabumi', 'kelaparan', 'rawan', 'penghujan', 'banjir', 'letusannya', 'pela

In [65]:
# df_test_unpreprocessed = pd.read_csv(PATH + bencana + "corpus/df_test_unprerocessed.csv")

# hasil_test_what = []

# for i in range(0, len(df_test_unpreprocessed)):
#     hasilWhat = []

#     for key in qeGabunganDelimiterWhat:
#         cariW = re.findall(key, df_test_unpreprocessed['desc'].iloc[i])

#         if cariW:
#             hasilWhat.append(key)
#     hasil_test_what.append(hasilWhat)

# hasil_test_what

df_test = pd.read_csv(PATH + bencana + "df_test_results.csv")

hasil_test_what = []

for i in range(0, len(df_test)):
    hasilWhat = []

    for key in qeGabunganDelimiterWhat:
        cariW = re.findall(key, df_test['desc'].iloc[i])

        if cariW:
            hasilWhat.append(key)
    hasil_test_what.append(hasilWhat)

hasil_test_what

[['bencana',
  'wilayah',
  'menyebabkan',
  'air',
  'banjir',
  'genangan',
  'menggenangi'],
 ['hulu', 'kebakaran', 'keterangan', 'warga', 'terbakar'],
 ['bencana',
  'cuaca',
  'ekstrem',
  'rusak',
  'daerah',
  'kecamatan',
  'hari',
  'warga',
  'tercatat',
  'rawan',
  'banjir',
  'diterjang',
  'longsor',
  'kencang',
  'angin'],
 ['kebakaran', 'hujan', 'warga', 'nyambar', 'petir', 'terbakar', 'angin'],
 ['bencana',
  'cuaca',
  'ekstrem',
  'mengguyur',
  'deras',
  'wilayah',
  'hujan',
  'hari',
  'tanah',
  'jalan',
  'banjir',
  'genangan',
  'kilat',
  'meteor',
  'petir',
  'longsor',
  'kencang',
  'angin'],
 ['bencana',
  'kebakaran',
  'keterangan',
  'manusia',
  'daerah',
  'hujan',
  'tanah',
  'jalan',
  'banjir',
  'erupsi',
  'gempa',
  'tertimpa',
  'bumi',
  'getaran',
  'terjal',
  'longsor',
  'kencang',
  'angin'],
 ['wilayah', 'air', 'banjir'],
 ['terpantau',
  'warga',
  'debit',
  'jalan',
  'air',
  'banjir',
  'kemacetan',
  'bahaya'],
 ['meluap',
  '

In [67]:
# df_test_unpreprocessed['qe_what_old']=hasil_test_what
# df_test_unpreprocessed

df_test['qe_what_new']=hasil_test_what

df_test['tp_old'] = 0
df_test['tn_old'] = 0
df_test['tp_new'] = 0
df_test['tn_new'] = 0

df_test


,title,date,desc,source,qe_what_old,qe_what_new,tp_old,tn_old,tp_new,tn_new
0,BPBD Waspadai Gelombang Pasang Air Laut di Pes...,"Rabu, 21 Des 2022 14:30 WIB",\-\r\n\r\nBPBD Lamongan mewaspadai pasang air ...,www.detik.com,"['bencana', 'banjir', 'desa', 'pesisir', 'pung...","[bencana, wilayah, menyebabkan, air, banjir, g...",0,0,0,0
1,"Bar Bekas Holywings di Tanjung Duren Terbakar,...","Senin, 05 Des 2022 20:19 WIB",\- Helen's Bar yang dahulu bernama Holywings T...,www.kompas.com,"['kebakaran', 'akibat', 'mengerahkan', 'warga'...","[hulu, kebakaran, keterangan, warga, terbakar]",0,0,0,0
2,"Angin Kencang Terjang Permukiman di Sidrap, 35...","Selasa, 03 Jan 2023 19:43 WIB","\-\r\n\r\n35 rumah di Kabupaten Sidrap, Sulawe...",www.detik.com,"['bencana', 'cuaca', 'banjir', 'kencang', 'kec...","[bencana, cuaca, ekstrem, rusak, daerah, kecam...",0,0,0,0
3,Kondisi Terkini 52 Rumah di Manggarai Hangus T...,"Minggu, 18 Des 2022 11:42 WIB",\-\r\n\r\nSebanyak 52 rumah dilanda kebakaran ...,www.detik.com,"['kebakaran', 'bantuan', 'mengungsi', 'jam', '...","[kebakaran, hujan, warga, nyambar, petir, terb...",0,0,0,0
4,"BMKG Rilis Peringatan Dini, Hujan Deras dan An...","Jumat, 09 Des 2022 13:44 WIB","\- Badan Meteorologi, Klimatologi dan Geofisik...",www.kompas.com,"['bencana', 'cuaca', 'banjir', 'meteorologi', ...","[bencana, cuaca, ekstrem, mengguyur, deras, wi...",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
273,Geolog UPN soal Dentuman Misterius di Gunungki...,"Jumat, 23 Des 2022 15:30 WIB",\-\r\n\r\nSuara dentuman misterius terdengar d...,www.detik.com,"['wilayah', 'gempa', 'runtuh', 'terkikis', 'fe...","[wilayah, tanah, menyebabkan, air, gempa, bumi...",0,0,0,0
274,"Mangsa Ayam, Piton 2,5 Meter 'Ditaklukan' Petu...","Kamis, 22 Des 2022 22:31 WIB",\-\r\n\r\nKampung Babakanbaru RT 03/06 Desa Pa...,www.detik.com,"['kebakaran', 'wilayah', 'lokasi', 'warga', 'd...","[kebakaran, wilayah, pemukiman, warga, kelapar...",0,0,0,0
275,"Jokowi Resmikan Bendungan Ciawi dan Sukamahi, ...","Jumat, 23 Des 2022 10:00 WIB",Presiden Joko Widodo meresmikan Bendungan Ciaw...,www.kompas.com,"['banjir', 'terdampak', 'waduk']","[pembangunan, langganan, air, banjir, tanggul]",0,0,0,0
276,"Cerita Siswa Terdampak Rob di Bonang Demak, ke...","Selasa, 13 Des 2022 10:23 WIB",\-\r\n\r\nBanjir rob yang terjadi di Kecamatan...,www.detik.com,"['banjir', 'wilayah', 'kabupaten', 'hari', 'lo...","[wilayah, kabupaten, hari, tanah, sambungnya, ...",0,0,0,0


In [78]:
# df_test_unpreprocessed.to_csv(PATH + bencana + "df_test_results.csv" , index = None, header=True)
df_test = df_test.drop(['source'], axis=1)
df_test.to_csv(PATH + bencana + "df_test_results_appended.csv" , index = None, header=True)

## LDA

In [14]:
import gensim
from gensim import corpora
from collections import Counter
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [15]:
def remove_number(text):
    return  re.sub(r"\d+", "", text)

In [163]:
new_bencana = old_bencana[['title', 'desc']]
new_bencana['desc'] = new_bencana['desc'].apply(remove_number)
new_bencana['desc'] = new_bencana['desc'].apply(word_tokenize)
new_bencana['desc'] = [[token for token in doc if len(token) > 2] for doc in new_bencana['desc']]
new_bencana['title'] = new_bencana['title'].apply(remove_number)
new_bencana['title'] = new_bencana['title'].apply(word_tokenize)
new_bencana['title'] = [[token for token in doc if len(token) > 2] for doc in new_bencana['title']]
new_bencana['fdist'] = new_bencana['desc'].apply(FreqDist)

print(len(new_bencana))
print(new_bencana.head(2))

print('Frequency Tokens : \n') 
print(new_bencana['fdist'].head().apply(lambda x : x.most_common()))

In [166]:
from gensim.models import Phrases
bigram = Phrases(new_bencana['desc'], min_count=20)
for idx in range(len(new_bencana['desc'])):
    for token in bigram[new_bencana['desc'].iloc[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            new_bencana['desc'].iloc[idx].append(token)
            word1, word2 = token.split("_")
            if word1 not in ('kebakaran', 'banjir', 'gempa', 'longsor', 'banjir', 'angin'):
                new_bencana['desc'].iloc[idx].remove(word1)
            if word2 not in ('kebakaran', 'banjir', 'gempa', 'longsor', 'banjir', 'angin'):
                new_bencana['desc'].iloc[idx].remove(word2)
            # print(token)

In [167]:
dictionary = corpora.Dictionary(new_bencana['desc'])
dictionary.filter_extremes(no_below=30, no_above=0.5)
print(dictionary)

corpus = [dictionary.doc2bow(berita) for berita in new_bencana['desc']]

Dictionary<1656 unique tokens: ['api', 'arah', 'banjir', 'banjir_menggenangi', 'depannya']...>


In [168]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 4 # jumlah topik yang akan di extract
number_words = 30 # jumlah kata per topik

In [169]:
import os
from gensim.test.utils import datapath
the_path = PATH + bencana + "model/lda_model" + "_" + str(total_topics) + "_" + str(number_words)
if not os.path.exists(the_path):
    os.mkdir(the_path)
temp_file = datapath(the_path + "/lda_model")
#if new
lda_model = Lda(corpus, num_topics=total_topics, id2word=dictionary, passes=50, random_state=1)
lda_model.save(temp_file)
# load
from gensim import models
lda_model = models.ldamodel.LdaModel.load(temp_file)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

[(0,
  '0.029*"kebakaran" + 0.018*"longsor" + 0.009*"terbakar" + 0.009*"ditemukan" + 0.009*"api" + 0.007*"mobil" + 0.007*"pencarian" + 0.007*"peristiwa" + 0.006*"bangunan" + 0.006*"damkar" + 0.006*"polisi" + 0.005*"diduga" + 0.005*"tanah" + 0.005*"pasar" + 0.005*"tim" + 0.005*"jakarta" + 0.005*"meninggal" + 0.004*"unit" + 0.004*"rumahnya" + 0.004*"saksi" + 0.004*"berhasil" + 0.004*"pohon" + 0.004*"pria" + 0.004*"personel" + 0.004*"listrik" + 0.004*"jenazah" + 0.004*"pemadam_kebakaran" + 0.004*"sampah" + 0.004*"mengaku" + 0.004*"pelaku"'),
 (1,
  '0.088*"banjir" + 0.016*"sungai" + 0.016*"jakarta" + 0.014*"semarang" + 0.009*"genangan" + 0.008*"bpbd" + 0.008*"jembatan" + 0.008*"titik" + 0.007*"ketinggian" + 0.007*"surut" + 0.006*"bandung" + 0.006*"terendam_banjir" + 0.006*"terdampak" + 0.006*"terdampak_banjir" + 0.006*"kali" + 0.006*"banjir_bandang" + 0.006*"hujan" + 0.006*"akses" + 0.005*"mengungsi" + 0.005*"pembangunan" + 0.005*"dki_jakarta" + 0.005*"jawa" + 0.005*"kemarin" + 0.005*"rob

In [186]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False, num_topics=total_topics, num_words=number_words)
# topic = lda_model.show_topics(num_topics=total_topics, num_words=number_words)
# print((topics[0]))
# print((topic[0]))
data_flat = [w for w_list in new_bencana['desc'] for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'id', 'importance', 'word_count']) 
print(df_imp_wcount)

          word  id  importance  word_count
0    kebakaran   0    0.029152        1595
1      longsor   0    0.017610        1358
2     terbakar   0    0.008810         435
3    ditemukan   0    0.008799         427
4          api   0    0.008587         509
..         ...  ..         ...         ...
115     wisata   3    0.005074         269
116   lumajang   3    0.005073         176
117      natal   3    0.005014         170
118    tanggal   3    0.005009         248
119   makassar   3    0.004645         463

[120 rows x 4 columns]


In [171]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=new_bencana['desc']):
    # Init output
    sent_topics_df = pd.DataFrame()
    # print(len(ldamodel[corpus]))

    dom = []
    perc = []
    keywords = []
    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # print(len(row))

        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num, topn=number_words)
                topic_keywords = ", ".join([word for word, prop in wp])
                dom.append(int(topic_num))
                perc.append(round(prop_topic,4))
                keywords.append(topic_keywords)
                # sent_topics_df = pd.concat([sent_topics_df, pd.DataFrame([pd.Series([int(topic_num), round(prop_topic,4), topic_keywords])])], ignore_index=True)
                # sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    # print(len(sent_topics_df))
    sent_topics_df['Dominant_Topic'] = dom
    sent_topics_df['Perc_Contribution'] = perc
    sent_topics_df['Topic_Keywords'] = keywords
    # sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    # sent_topics_df['Texts']= texts
    # print(len(sent_topics_df))
    return(sent_topics_df)

In [187]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=new_bencana['desc'])
df_topic_sents_keywords['Texts'] = 0
df_topic_sents_keywords['Title'] = 0

In [188]:
df_dominant_topic = df_topic_sents_keywords.reset_index()
# texts = []
for i in range(len(new_bencana['desc'])):
    # print(new_bencana['desc'].iloc[i])
    # texts.append(new_bencana['desc'].iloc[i])
    df_dominant_topic['Texts'].iloc[i] = new_bencana['desc'].iloc[i]
    df_dominant_topic['Title'].iloc[i] = new_bencana['title'].iloc[i]
# df_dominant_topic['Texts'] = texts
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Title']

# df_topic_sents_keywords['Texts'].isna().value_counts()
df_dominant_topic.head(5)
df_dominant_topic.tail(5)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Title
2768,2768,1,0.9957,"banjir, sungai, jakarta, semarang, genangan, b...","[pantai, jawa, pantura, jawa, banjir, perganti...","[bpbd, laporkan, jateng, tergenang, banjir, te..."
2769,2769,1,0.8189,"banjir, sungai, jakarta, semarang, genangan, b...","[semarang, kaline, banjir, lirik, lagu, berjud...","[banjir, semarang, dipisahkan]"
2770,2770,2,0.9927,"gempa, gempa_bumi, cianjur, bantuan, bmkg, ban...","[memperbarui, laporan, gempa, mengguncang, gem...","[jayapura, rasakan, gempa, susulan]"
2771,2771,2,0.8773,"gempa, gempa_bumi, cianjur, bantuan, bmkg, ban...","[gempa, magnitudo, dilaporkan, gempa, puluhan,...","[breaking, news, gempa, guncang, jayapura, pap..."
2772,2772,0,0.7579,"kebakaran, longsor, terbakar, ditemukan, api, ...","[mahasiswa, melintasi, banjir, merayakan, acar...","[mahasiswa, semarang, tersengat, listrik, lint..."


In [189]:
df_dominant_topic.to_csv(PATH + bencana + 'df_dominant_topic.csv')

In [200]:
pyLDAvis.enable_notebook()

In [194]:
from datetime import datetime
now = datetime.now() # current date and time
date_time = now.strftime("%m-%d-%Y_%H-%M-%S")
print("date and time:",date_time)
LDAvis_html = 'ldavis_prepared_'+ str(total_topics)+"_"+str(number_words)+"_"+str(date_time)
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
with open(PATH + bencana + LDAvis_html, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

date and time: 06-23-2023_14-30-24


In [195]:
with open(PATH + bencana + LDAvis_html, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [196]:
pyLDAvis.save_html(LDAvis_prepared, PATH + bencana + LDAvis_html +'.html')

In [423]:
# df_imp_wcount.sort_values(by=['word_count'], ascending=False)
# df_dominant_topic['Keywords'].iloc[2771]
# lda_model.show_topic(1, topn=15)

In [174]:
lda_what_bencana = []
word_topics = set(df_imp_wcount['word'].values)
for i in range(len(df_dominant_topic)):
    val = []
    for n, key in enumerate(word_topics):
        if key in df_dominant_topic['Text'].iloc[i]:
            val.append(key)
    lda_what_bencana.append(val)

print(lda_what_bencana)

[['jalur', 'genangan', 'pasar', 'semarang', 'api', 'jakarta', 'jawa', 'banjir'], ['surut', 'genangan', 'terendam_banjir', 'bpbd', 'titik', 'gunung', 'terdampak_banjir', 'kemarin', 'banjir', 'mengungsi', 'ketinggian'], ['pemadam_kebakaran', 'kebakaran', 'terbakar', 'listrik', 'sekolah', 'aktivitas', 'rumahnya', 'bpbd', 'api', 'diduga', 'peristiwa', 'unit', 'ketinggian'], ['sampah', 'pemerintah', 'sungai', 'potensi', 'pohon', 'genangan', 'rusak', 'terendam_banjir', 'bpbd', 'longsor', 'jembatan', 'kemarin', 'banjir', 'kali', 'ketinggian'], ['laut', 'bantuan', 'sungai', 'penanganan', 'mengaku', 'alam', 'terendam_banjir', 'rumahnya', 'melanda', 'bpbd', 'banjir_bandang', 'titik', 'terdampak', 'terdampak_banjir', 'unit', 'banjir'], ['sampah', 'sungai', 'jalur', 'kendaraan', 'surut', 'genangan', 'hujan', 'polisi', 'titik', 'jembatan', 'banjir', 'kali'], ['laut', 'sungai', 'surut', 'sekolah', 'terendam_banjir', 'aktivitas', 'bangunan', 'rob', 'titik', 'terdampak', 'jawa', 'terdampak_banjir', 'b

In [175]:
lda_title_what = []
word_topics = set(df_imp_wcount['word'].values)
for i in range(len(df_dominant_topic)):
    val = []
    for n, key in enumerate(word_topics):
        if key in df_dominant_topic['Title'].iloc[i]:
            val.append(key)
    lda_title_what.append(val)

print(lda_title_what)

[['semarang', 'jakarta', 'banjir'], ['titik', 'banjir'], ['terbakar'], ['bpbd', 'longsor', 'tanah', 'banjir'], ['banjir'], ['kendaraan', 'banjir'], ['rob', 'banjir', 'ketinggian'], ['cianjur', 'rusak', 'magnitudo', 'garut', 'gempa'], ['berpotensi', 'magnitudo', 'gempa'], [], ['hujan', 'banjir'], ['damkar'], ['terbakar', 'personel', 'damkar'], ['jakarta', 'banjir'], ['cianjur', 'hujan', 'gempa'], ['terbakar', 'pasar'], ['bantuan', 'cianjur', 'gempa'], ['bantuan', 'cianjur', 'gempa'], ['banjir'], ['bantuan', 'cianjur', 'gempa'], ['kebakaran', 'pasar', 'tim'], ['banjir'], ['angin', 'rusak'], ['kebakaran', 'pasar', 'tim'], ['kebakaran', 'damkar', 'mobil'], ['terbakar'], ['terbakar'], ['angin', 'kencang'], ['angin', 'kencang'], ['ditemukan', 'banjir'], ['potensi', 'angin', 'hujan', 'kencang', 'bmkg', 'cuaca'], ['banjir'], ['angin', 'hujan', 'jakarta', 'kencang', 'cuaca'], ['angin', 'hujan', 'jakarta', 'kencang'], ['angin', 'hujan', 'jakarta', 'kencang', 'cuaca'], ['angin', 'hujan', 'jakarta

In [176]:
for n, word_list in enumerate(lda_title_what):
    val = []
    for word in word_list:
        if word in lda_what_bencana[n]:
            continue
        val.append(word)
    lda_what_bencana[n].extend(val)
    # if val:
    #     print(n)
    #     print(val)

print(lda_what_bencana)

[['jalur', 'genangan', 'pasar', 'semarang', 'api', 'jakarta', 'jawa', 'banjir'], ['surut', 'genangan', 'terendam_banjir', 'bpbd', 'titik', 'gunung', 'terdampak_banjir', 'kemarin', 'banjir', 'mengungsi', 'ketinggian'], ['pemadam_kebakaran', 'kebakaran', 'terbakar', 'listrik', 'sekolah', 'aktivitas', 'rumahnya', 'bpbd', 'api', 'diduga', 'peristiwa', 'unit', 'ketinggian'], ['sampah', 'pemerintah', 'sungai', 'potensi', 'pohon', 'genangan', 'rusak', 'terendam_banjir', 'bpbd', 'longsor', 'jembatan', 'kemarin', 'banjir', 'kali', 'ketinggian', 'tanah'], ['laut', 'bantuan', 'sungai', 'penanganan', 'mengaku', 'alam', 'terendam_banjir', 'rumahnya', 'melanda', 'bpbd', 'banjir_bandang', 'titik', 'terdampak', 'terdampak_banjir', 'unit', 'banjir'], ['sampah', 'sungai', 'jalur', 'kendaraan', 'surut', 'genangan', 'hujan', 'polisi', 'titik', 'jembatan', 'banjir', 'kali'], ['laut', 'sungai', 'surut', 'sekolah', 'terendam_banjir', 'aktivitas', 'bangunan', 'rob', 'titik', 'terdampak', 'jawa', 'terdampak_ba

In [177]:
idxs = []
idxs_tp = []
idxs_tn = []
total = 0
total_tp = 0
total_tn = 0
for i in range(len(lda_what_bencana)):
    flag = True
    deflag = True
    for token in lda_what_bencana[i]:
        if token in ('kebakaran', 'banjir', 'gempa', 'gempa_bumi', 'longsor', 'banjir_rob', 'angin_kencang', 'angin', 'cuaca_ekstrem', 'hujan'):
            flag = False
    for token in df_dominant_topic['Text'].iloc[i]:
        if token in ('kebakaran', 'banjir', 'gempa', 'gempa_bumi', 'longsor', 'banjir_rob', 'angin_kencang', 'angin', 'cuaca_ekstrem', 'hujan'):
            deflag = False
    for token in df_dominant_topic['Title'].iloc[i]:
        if token in ('kebakaran', 'banjir', 'gempa', 'gempa_bumi', 'longsor', 'banjir_rob', 'angin_kencang', 'angin', 'cuaca_ekstrem', 'hujan'):
            deflag = False
    if flag != deflag:
        total += 1
        idxs.append(i)
    elif flag == deflag and flag == False:
        total_tp += 1
        idxs_tp.append(i)
    elif flag == deflag and flag == True:
        total_tn += 1
        idxs_tn.append(i)
print(total)
print(total_tp)
print(total_tn)

0
2679
94


In [178]:
lda_check = []
for word in ('kebakaran', 'banjir', 'gempa', 'gempa_bumi', 'longsor', 'banjir_rob', 'angin_kencang', 'angin', 'cuaca_ekstrem', 'hujan'):
    if word not in df_imp_wcount['word'].values:
        lda_check.append(word)
print(lda_check)

[]


In [156]:
print(idxs_tn)

[11, 137, 138, 140, 141, 152, 156, 216, 279, 352, 454, 535, 536, 716, 753, 755, 760, 775, 835, 840, 867, 869, 888, 893, 894, 895, 906, 1114, 1123, 1133, 1254, 1259, 1267, 1284, 1339, 1416, 1417, 1424, 1455, 1464, 1473, 1496, 1535, 1603, 1607, 1612, 1624, 1631, 1640, 1660, 1681, 1714, 1729, 1764, 1765, 1776, 1777, 1825, 1829, 1834, 1863, 1912, 1938, 1975, 1976, 1988, 2008, 2022, 2028, 2031, 2037, 2068, 2075, 2077, 2083, 2112, 2139, 2212, 2232, 2240, 2325, 2417, 2423, 2448, 2449, 2454, 2462, 2629, 2652, 2670, 2680, 2733, 2748, 2754]


In [179]:
# text  = preprocessing(old_bencana['desc'][old_bencana['desc'].str.find('hari') != -1].iloc[0])
# text
i = idxs_tn[0]
print(df_dominant_topic['Keywords'].iloc[i])
print(type(df_dominant_topic['Keywords'].iloc[i]))
print(df_dominant_topic['Text'].iloc[i])
print(type(df_dominant_topic['Text'].iloc[i]))
print(df_dominant_topic['Keywords'].iloc[i].split(", "))
print(lda_what_bencana[i])


kebakaran, longsor, terbakar, ditemukan, api, mobil, pencarian, peristiwa, bangunan, damkar, polisi, diduga, tanah, pasar, tim, jakarta, meninggal, unit, rumahnya, saksi, berhasil, pohon, pria, personel, listrik, jenazah, pemadam_kebakaran, sampah, mengaku, pelaku
<class 'str'>
['kebonsari', 'sumbersari', 'rumahnya', 'balita', 'terpeleset', 'bermain', 'terjatuh', 'damkar', 'pencarian', 'komandan', 'regu', 'damkar', 'jember', 'dwi', 'atmoko', 'via', 'telepon', 'laporan', 'dwi', 'mengirim', 'personel', 'pencarian', 'mencari', 'balita', 'menyisir', 'selokan', 'digenangi', 'selokan', 'ketemu', 'selokan', 'saluran', 'perumahan', 'berbeda', 'kebingungan', 'mencari', 'lakukan', 'pengecekan', 'damkar', 'pencarian', 'menemukan', 'menyusuri', 'akhirya', 'mencoba', 'mencari', 'waduk', 'persawahan', 'bermuara', 'dibantu', 'pencarian', 'menjelang', 'pencarian', 'membuahkan', 'hasil', 'beristirahat', 'berselang', 'menemukan', 'saluran', 'ditemukan', 'bernyawa', 'jenazah', 'nyangkut', 'rerumputan', '

In [180]:
before_what_bencana = old[old['berita_topik_id'] == 1]
before_what_bencana = before_what_bencana[['berita_title', 'qe_what', 'berita_desc']]
before_what_bencana = before_what_bencana.dropna(subset=['berita_desc'])
before_what_bencana.sort_values('berita_desc', inplace=True)
before_what_bencana.drop_duplicates(subset='berita_desc', keep=False, inplace=True)
# print(before_what_bencana)
# print(before_what_bencana['qe_what'].iloc[i])
# print(before_what_bencana['berita_title'].iloc[i])
print(before_what_bencana['berita_desc'].iloc[i])

 KR (2), warga Kelurahan Kebonsari, Kecamatan Sumbersari, Kabupaten Jember,
Jawa Timur, terseret arus saluran air di depan rumahnya, Selasa (6/12/2022).

Balita tersebut terpeleset saat bermain hingga terjatuh ke saluran air.
Petugas Damkar mendapatkan informasi anak yang terseret arus tersebut dan
segera melakukan pencarian.

Kami dapat informasi dari warga ada anak terseret arus, kata Komandan Regu
Damkar Jember Dwi Atmoko kepada _Kompas.com_ via telepon, Rabu (7/12/2022).

Setelah mendapat laporan, Dwi mengirim lima personel melakukan pencarian. Para
petugas mencari balita itu dengan menyisir selokan yang digenangi air.

Mereka juga masuk ke gorong-gorong selokan, namun belum ketemu, tambah dia.

Menurut dia, ada tiga gorong-gorong di selokan tersebut. Yakni saluran air
yang menuju perumahan berbeda. Hal itu membuat petugas kebingungan dalam
mencari korban.

Namun semua gorong-gorong itu kami lakukan pengecekan, tambah dia.

Para petugas damkar melakukan pencarian di gorong-gorong s

In [185]:
total_empty = 0
empty_val = []
with open(PATH + bencana + 'ture_negative_4.txt', 'w', encoding='UTF8') as f:
    nl = "\n"
    dashes = '--------------------------------------'
    for x in idxs_tn:
        f.write(dashes + nl)
        f.write(before_what_bencana['berita_title'].iloc[x] + nl)
        f.write(before_what_bencana['qe_what'].iloc[x] + nl)
        f.write(str(lda_what_bencana[x]) + nl)
        # print(before_what_bencana['berita_title'].iloc[x])
        # print(before_what_bencana['qe_what'].iloc[x])
        # print(lda_what_bencana[x])

        if len(lda_what_bencana[x])<3:
            total_empty += 1
            empty_val.append(x)

print(total_empty)
print(empty_val)


11
[352, 716, 869, 893, 1424, 1455, 1464, 1729, 2629, 2652, 2754]


In [182]:
hitung = Counter()
for i in idxs_tn:
    # hitung.update(lda_what_bencana[i])
    hitung.update(lda_title_what[i])

In [183]:
hitung

Counter({'tanah': 10,
         'rusak': 6,
         'potensi': 5,
         'wisata': 4,
         'diduga': 3,
         'waspada': 3,
         'garut': 3,
         'jakarta': 3,
         'terbakar': 3,
         'pria': 2,
         'bpbd': 2,
         'makassar': 2,
         'meninggal': 2,
         'kapal': 2,
         'natal': 2,
         'pantai': 2,
         'sungai': 2,
         'pesisir': 2,
         'mobil': 2,
         'damkar': 1,
         'sampah': 1,
         'penanganan': 1,
         'fenomena': 1,
         'genangan': 1,
         'bumi': 1,
         'laut': 1,
         'rob': 1,
         'pasar': 1,
         'cuaca': 1,
         'bandung': 1,
         'gelombang': 1,
         'terdampak': 1,
         'kali': 1,
         'jenazah': 1,
         'tim': 1,
         'bali': 1,
         'ditemukan': 1,
         'berpotensi': 1,
         'jembatan': 1})

# Ekonomi

In [5]:
old_ekonomi = old[old['berita_topik_id'] == 2]
old_ekonomi = old_ekonomi[['berita_title', 'berita_date', 'berita_desc', 'berita_source']]
old_ekonomi.rename(columns = {'berita_title':'title', 'berita_date':'date', 'berita_desc':'desc', 'berita_source':'source'}, inplace=True)
print(old_ekonomi.shape)
old_ekonomi.head(2)

(3141, 4)


,title,date,desc,source
2005,UMKM Binaan BUMN Kolaborasi Beri Bantuan hingg...,"Sabtu, 24 Des 2022 08:17 WIB",\- Guna memperkenalkan pelayanan sosial berupa...,www.tribunnews.com
2006,Program Desa Brilian Diharapkan Menjadi Inspir...,"Kamis, 22 Des 2022 11:44 WIB",Pemberdayaan wilayah pedesaan menjadi isu yan...,www.tribunnews.com


## Preprocess

In [7]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# previous way

# nltk.download('punkt')
# nltk.download('stopwords')

excluded_words = ["tempat", "waktu", "hari"]
# # for lda ----
# excluded_words = []
# # ------------
NLTK_StopWords = stopwords.words('indonesian')
# print(NLTK_StopWords)
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali",
                      "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
# txt_stopword = pd.read_csv(PATH + ekonomi + "stopwords.txt", names= ["stopwords"], header = None)
# NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
# for lda ----
# txt_stopword = pd.read_csv(PATH + ekonomi + "stopwords_add_v1.txt", names= ["stopwords"], header = None)
# NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
# txt_stopword = pd.read_csv(PATH + ekonomi + "stopwords_add_v2.txt", names= ["stopwords"], header = None)
# NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
# txt_stopword = pd.read_csv(PATH + ekonomi + "stopwords_add_v3.txt", names= ["stopwords"], header = None)
# NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
# ------------
# convert stopword string to list & append additional stopword
# NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))

NLTK_StopWords = set(NLTK_StopWords)

In [9]:
def preprocessing(berita):
        s = str(berita)
        s = s.lower()
        s = s.replace('\n', ' ')
        s = s.replace('\r', ' ')
        # s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
        # for lda ----
        s = re.sub(r'[^a-zA-Z\s]', ' ', s)
        # ------------
        tokens = [token for token in s.split(" ") if token != ""]
        T = [t for t in tokens if (
            (t in excluded_words) or (t not in NLTK_StopWords))]
        return T

In [10]:
old_ekonomi = old_ekonomi.dropna(subset=['desc'])
old_ekonomi.sort_values('desc', inplace=True)
old_ekonomi.drop_duplicates(subset='desc', keep=False, inplace=True)
df_train, df_test = train_test_split(old_ekonomi,test_size=0.1)
print(len(df_train))

old_ekonomi.head()

2826


,title,date,desc,source
10619,Anggota Komisi XI DPR Dorong Pemerintah Lebih ...,"Senin, 02 Jan 2023 20:59 WIB",\r\n \r\n \r\n\- Anggota Komisi XI Dewan P...,www.tribunnews.com
4285,Analis Perkirakan Ekonomi China akan Bangkit L...,"Kamis, 22 Des 2022 19:24 WIB",\r\n \r\nPerekonomian China pada 2023 kemun...,www.tribunnews.com
4266,Ekonom Prediksi Jepang Bisa Masuk ke Jurang Re...,"Rabu, 07 Des 2022 16:19 WIB",\r\n \r\nPerusahaan riset ekonomi Capital E...,www.tribunnews.com
9981,KSP Klaim Perppu Cipta Kerja Dibuat atas Kepen...,"Kamis, 05 Jan 2023 15:21 WIB",\r\nKantor Staf Presiden (KSP) membantah ang...,www.kompas.com
4257,"Cerita Heni, Ibu Rumah Tangga yang Buka Kursus...","Minggu, 25 Des 2022 13:07 WIB",\r\nPerempuan yang telah menikah dan memilik...,www.kompas.com


In [ ]:
df_train.to_csv(PATH + ekonomi + "corpus/df_train_unprerocessed.csv" , index = None, header=True)
df_test.to_csv(PATH + ekonomi + "corpus/df_test_unprerocessed.csv" , index = None, header=True)

In [ ]:
desc_text_train_unpreprocessed = []
desc_text_test_unpreprocessed = []

for x in range(0, df_train.shape[0]):
  desc_text_train_unpreprocessed.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test_unpreprocessed.append(df_test.iloc[x, -2])

In [ ]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test_unpreprocessed, PATH + ekonomi + "corpus/model/desc_text_test_unpreprocessed.pkl")
joblib.dump(desc_text_train_unpreprocessed, PATH + ekonomi + "corpus/model/desc_text_train_unpreprocessed.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
old_ekonomi.to_csv(PATH + ekonomi + 'corpus/dataset/df_total_unpreprocessed.csv', index=False)
df_test.to_csv(PATH + ekonomi + 'corpus/dataset/df_test_unpreprocessed.csv', index=False)
df_train.to_csv(PATH + ekonomi + 'corpus/dataset/df_train_unpreprocessed.csv', index=False)

In [11]:
for x in range(0, old_ekonomi.shape[0]):
    text = preprocessing(old_ekonomi.iloc[x, -2])
    old_ekonomi.iloc[x, -2] = (' '.join(text))

for x in range(0, old_ekonomi.shape[0]):
    text = preprocessing(old_ekonomi.iloc[x, -4])
    old_ekonomi.iloc[x, -4] = (' '.join(text)) 
old_ekonomi.head()

,title,date,desc,source
10619,anggota komisi xi dpr dorong pemerintah perhat...,"Senin, 02 Jan 2023 20:59 WIB",anggota komisi xi dewan perwakilan rakyat dpr ...,www.tribunnews.com
4285,analis perkirakan ekonomi china bangkit cepat ...,"Kamis, 22 Des 2022 19:24 WIB",perekonomian china perekonomian negara beijing...,www.tribunnews.com
4266,ekonom prediksi jepang masuk jurang resesi did...,"Rabu, 07 Des 2022 16:19 WIB",perusahaan riset ekonomi capital economics per...,www.tribunnews.com
9981,ksp klaim perppu cipta kerja kepentingan peker...,"Kamis, 05 Jan 2023 15:21 WIB",kantor staf presiden ksp membantah anggapan pe...,www.kompas.com
4257,cerita heni rumah tangga buka kursus online ba...,"Minggu, 25 Des 2022 13:07 WIB",perempuan menikah memiliki anak seringkali mem...,www.kompas.com


In [ ]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

In [ ]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, PATH + ekonomi + "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, PATH + ekonomi + "corpus/model/desc_text_train_new.pkl") #INI

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
old_ekonomi.to_csv(PATH + ekonomi + 'corpus/dataset/df_total_new.csv', index=False) #INI
df_test.to_csv(PATH + ekonomi + 'corpus/dataset/df_test.csv', index=False)
df_train.to_csv(PATH + ekonomi + 'corpus/dataset/df_train.csv', index=False)

In [ ]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

In [ ]:
joblib.dump(tfidf_vectorizer, PATH + ekonomi + "corpus/vectorizer/vectorizer_new.pkl") #INI
joblib.dump(tfidf_test_matrix, PATH + ekonomi + "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, PATH + ekonomi + "corpus/matrix/tfidf_train_new.pkl") #INI

## YAKE BERT TFIDF

## MAIN COURSE

## LDA

In [6]:
import gensim
from gensim import corpora
from collections import Counter
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [12]:
def remove_number(text):
    return  re.sub(r"\d+", "", text)

In [17]:
new_ekonomi = old_ekonomi[['title', 'desc']]
new_ekonomi['desc'] = new_ekonomi['desc'].apply(remove_number)
new_ekonomi['desc'] = new_ekonomi['desc'].apply(word_tokenize)
new_ekonomi['desc'] = [[token for token in doc if len(token) > 2] for doc in new_ekonomi['desc']]
new_ekonomi['title'] = new_ekonomi['title'].apply(remove_number)
new_ekonomi['title'] = new_ekonomi['title'].apply(word_tokenize)
new_ekonomi['title'] = [[token for token in doc if len(token) > 2] for doc in new_ekonomi['title']]
new_ekonomi['fdist'] = new_ekonomi['desc'].apply(FreqDist)

print(len(new_ekonomi))
print(new_ekonomi.head(2))

print('Frequency Tokens : \n') 
print(new_ekonomi['fdist'].head().apply(lambda x : x.most_common()))

3141
                                                   title  \
10619  [anggota, komisi, dpr, dorong, pemerintah, per...   
4285   [analis, perkirakan, ekonomi, china, bangkit, ...   

                                                    desc  \
10619  [anggota, komisi, dewan, perwakilan, rakyat, d...   
4285   [perekonomian, china, perekonomian, negara, be...   

                                                   fdist  
10619  {'anggota': 1, 'komisi': 1, 'dewan': 1, 'perwa...  
4285   {'perekonomian': 2, 'china': 20, 'negara': 12,...  
Frequency Tokens : 

10619    [(umkm, 21), (aday, 14), (pemerintah, 7), (ind...
4285     [(china, 20), (negara, 12), (persen, 10), (eko...
4266     [(persen, 10), (jepang, 8), (resesi, 6), (thie...
9981     [(pemerintah, 12), (perppu, 12), (kerja, 10), ...
4257     [(heni, 14), (kursus, 10), (anak, 9), (teknolo...
Name: fdist, dtype: object


In [18]:
from gensim.models import Phrases
bigram = Phrases(new_ekonomi['desc'], min_count=20)
for idx in range(len(new_ekonomi['desc'])):
    for token in bigram[new_ekonomi['desc'].iloc[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            new_ekonomi['desc'].iloc[idx].append(token)
            word1, word2 = token.split("_")
            # if word1 not in ('kebakaran', 'banjir', 'gempa', 'longsor', 'banjir', 'angin'):
            #     new_ekonomi['desc'].iloc[idx].remove(word1)
            # if word2 not in ('kebakaran', 'banjir', 'gempa', 'longsor', 'banjir', 'angin'):
            #     new_ekonomi['desc'].iloc[idx].remove(word2)
            # print(token)

In [19]:
dictionary = corpora.Dictionary(new_ekonomi['desc'])
dictionary.filter_extremes(no_below=35, no_above=0.5)
print(dictionary)

corpus = [dictionary.doc2bow(berita) for berita in new_ekonomi['desc']]

Dictionary<2629 unique tokens: ['acara', 'achmad', 'akrab', 'akses', 'anggota']...>


In [20]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 4 # jumlah topik yang akan di extract
number_words = 30 # jumlah kata per topik

In [21]:
import os
from gensim.test.utils import datapath
the_path = PATH + ekonomi + "model/lda_model" + "_" + str(total_topics) + "_" + str(number_words)
if not os.path.exists(the_path):
    os.mkdir(the_path)
temp_file = datapath(the_path + "/lda_model")
#if new
lda_model = Lda(corpus, num_topics=total_topics, id2word=dictionary, passes=50, random_state=1)
lda_model.save(temp_file)
# load
from gensim import models
lda_model = models.ldamodel.LdaModel.load(temp_file)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

[(0,
  '0.010*"kota" + 0.009*"masyarakat" + 0.009*"orang" + 0.008*"anak" + 0.007*"kabupaten" + 0.007*"desa" + 0.007*"hari" + 0.006*"kegiatan" + 0.006*"program" + 0.006*"warga" + 0.006*"jalan" + 0.005*"video" + 0.005*"jawa" + 0.005*"jakarta" + 0.005*"rumah" + 0.005*"daerah" + 0.005*"pendidikan" + 0.005*"salah" + 0.005*"memiliki" + 0.005*"gambas" + 0.004*"ketua" + 0.004*"gambas_video" + 0.004*"barat" + 0.004*"bali" + 0.004*"ganjar" + 0.004*"universitas" + 0.004*"kepala" + 0.004*"sosial" + 0.004*"keluarga" + 0.003*"tempat"'),
 (1,
  '0.021*"negara" + 0.016*"ekspor" + 0.015*"jokowi" + 0.014*"harga" + 0.011*"pemerintah" + 0.010*"beras" + 0.008*"presiden" + 0.008*"minyak" + 0.008*"kerja" + 0.008*"perdagangan" + 0.008*"ton" + 0.008*"impor" + 0.007*"negeri" + 0.007*"pasal" + 0.007*"undang" + 0.007*"saham" + 0.006*"menteri" + 0.006*"juta" + 0.006*"miliar" + 0.006*"perppu" + 0.005*"hukum" + 0.005*"cipta" + 0.004*"korupsi" + 0.004*"produksi" + 0.004*"pasar" + 0.004*"nikel" + 0.004*"nomor" + 0.004

In [22]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False, num_topics=total_topics, num_words=number_words)
# topic = lda_model.show_topics(num_topics=total_topics, num_words=number_words)
# print((topics[0]))
# print((topic[0]))
data_flat = [w for w_list in new_ekonomi['desc'] for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'id', 'importance', 'word_count']) 
print(df_imp_wcount)

             word  id  importance  word_count
0            kota   0    0.009851        1848
1      masyarakat   0    0.009316        3671
2           orang   0    0.009008        2143
3            anak   0    0.007558        1389
4       kabupaten   0    0.007077        1237
..            ...  ..         ...         ...
115    pembiayaan   3    0.003727         661
116  pelaku_usaha   3    0.003699         577
117         mikro   3    0.003546         552
118      industri   3    0.003542        1462
119     mendorong   3    0.003437        1115

[120 rows x 4 columns]


In [23]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=new_ekonomi['desc']):
    # Init output
    sent_topics_df = pd.DataFrame()
    # print(len(ldamodel[corpus]))

    dom = []
    perc = []
    keywords = []
    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # print(len(row))

        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num, topn=number_words)
                topic_keywords = ", ".join([word for word, prop in wp])
                dom.append(int(topic_num))
                perc.append(round(prop_topic,4))
                keywords.append(topic_keywords)
                # sent_topics_df = pd.concat([sent_topics_df, pd.DataFrame([pd.Series([int(topic_num), round(prop_topic,4), topic_keywords])])], ignore_index=True)
                # sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    # print(len(sent_topics_df))
    sent_topics_df['Dominant_Topic'] = dom
    sent_topics_df['Perc_Contribution'] = perc
    sent_topics_df['Topic_Keywords'] = keywords
    # sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    # sent_topics_df['Texts']= texts
    # print(len(sent_topics_df))
    return(sent_topics_df)

In [24]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=new_ekonomi['desc'])
df_topic_sents_keywords['Texts'] = 0
df_topic_sents_keywords['Title'] = 0

In [25]:
df_dominant_topic = df_topic_sents_keywords.reset_index()
# texts = []
for i in range(len(new_ekonomi['desc'])):
    # print(new_ekonomi['desc'].iloc[i])
    # texts.append(new_ekonomi['desc'].iloc[i])
    df_dominant_topic['Texts'].iloc[i] = new_ekonomi['desc'].iloc[i]
    df_dominant_topic['Title'].iloc[i] = new_ekonomi['title'].iloc[i]
# df_dominant_topic['Texts'] = texts
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Title']

# df_topic_sents_keywords['Texts'].isna().value_counts()
df_dominant_topic.head(5)
df_dominant_topic.tail(5)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Title
3136,3136,0,0.7347,"kota, masyarakat, orang, anak, kabupaten, desa...","[menjelang, petang, aktivitas, lokasi, pengung...","[dapur, kreasi, upaya, kemensos, pulihkan, per..."
3137,3137,2,0.8730,"persen, pertumbuhan, negara, pemerintah, kebij...","[desember, pelaku, ekonomi, dunia, usaha, pebi...","[outlook, ekonomi]"
3138,3138,2,0.8929,"persen, pertumbuhan, negara, pemerintah, kebij...","[transisi, energi, misi, pengurangan, emisi, k...","[tantangan, transisi, energi, peluang, pertumb..."
3139,3139,3,0.4517,"umkm, bank, usaha, syariah, produk, bsi, digit...","[tua, mpr, bambang, soesatyo, asumsi, eksisten...","[bamsoet, dorong, koperasi, sokoguru, perekono..."
3140,3140,3,0.8631,"umkm, bank, usaha, syariah, produk, bsi, digit...","[tua, mpr, bambang, soesatyo, digitalisasi, um...","[bamsoet, tegaskan, digitalisasi, umkm]"


In [26]:
df_dominant_topic.to_csv(PATH + ekonomi + 'df_dominant_topic.csv')

In [34]:
pyLDAvis.enable_notebook()

In [28]:
from datetime import datetime
now = datetime.now() # current date and time
date_time = now.strftime("%m-%d-%Y_%H-%M-%S")
print("date and time:",date_time)
LDAvis_html = 'ldavis_prepared_'+ str(total_topics)+"_"+str(number_words)+"_"+str(date_time)
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
with open(PATH + ekonomi + LDAvis_html, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

date and time: 06-23-2023_19-01-24


In [ ]:
LDAvis_html = 'ldavis_prepared_4_30_06-23-2023_19-01-24'

In [36]:
with open(PATH + ekonomi + LDAvis_html, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [37]:
pyLDAvis.save_html(LDAvis_prepared, PATH + ekonomi + LDAvis_html +'.html')

In [41]:
pyLDAvis.show(LDAvis_prepared)
# pyLDAvis.display(LDAvis_prepared, template_type='notebook')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 pyLDAvis.show(LDAvis_prepared)                                                               │
│   2 # pyLDAvis.display(LDAvis_prepared, template_type='notebook')                                │
│   3                                                                                              │
│                                                                                                  │
│ c:\Users\Archy\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_display.py:26 │
│ 2 in show                                                                                        │
│                                                                                                  │
│   259 │   │   kwargs['d3_url'] = urls.D3_URL                                                     │
│   260 │   │   kwargs['ldavis_css_url'] = urls.LDAVIS_CSS_URL                                     │
│   261 │   │   files = {'/LDAvis.js': ["text/javascript", open(urls.LDAVIS_LOCAL, 'r').read()],   │
│ ❱ 262 │   │   │   │    '/LDAvis.css': ["text/css", open(urls.LDAVIS_CSS_URL, 'r').read()],       │
│   263 │   │   │   │    '/d3.js': ["text/javascript", open(urls.D3_URL, 'r').read()]}             │
│   264 │   html = prepared_data_to_html(data, **kwargs)                                           │
│   265 │   serve(html, ip=ip, port=port, n_retries=n_retries, files=files,                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: [Errno 22] Invalid argument: 
'https://cdn.jsdelivr.net/gh/bmabey/pyLDAvis@3.4.0/pyLDAvis/js/ldavis.v1.0.0.css'

In [ ]:
lda_what_ekonomi = []
word_topics = set(df_imp_wcount['word'].values)
for i in range(len(df_dominant_topic)):
    val = []
    for n, key in enumerate(word_topics):
        if key in df_dominant_topic['Text'].iloc[i]:
            val.append(key)
    lda_what_ekonomi.append(val)

print(lda_what_ekonomi)

In [ ]:
lda_title_what = []
word_topics = set(df_imp_wcount['word'].values)
for i in range(len(df_dominant_topic)):
    val = []
    for n, key in enumerate(word_topics):
        if key in df_dominant_topic['Title'].iloc[i]:
            val.append(key)
    lda_title_what.append(val)

print(lda_title_what)

In [ ]:
for n, word_list in enumerate(lda_title_what):
    val = []
    for word in word_list:
        if word in lda_what_ekonomi[n]:
            continue
        val.append(word)
    lda_what_ekonomi[n].extend(val)
    # if val:
    #     print(n)
    #     print(val)

print(lda_what_ekonomi)

In [ ]:
idxs = []
idxs_tp = []
idxs_tn = []
total = 0
total_tp = 0
total_tn = 0
for i in range(len(lda_what_ekonomi)):
    flag = True
    deflag = True
    for token in lda_what_ekonomi[i]:
        if token in ('kebakaran', 'banjir', 'gempa', 'gempa_bumi', 'longsor', 'banjir_rob', 'angin_kencang', 'angin', 'cuaca_ekstrem', 'hujan'):
            flag = False
    for token in df_dominant_topic['Text'].iloc[i]:
        if token in ('kebakaran', 'banjir', 'gempa', 'gempa_bumi', 'longsor', 'banjir_rob', 'angin_kencang', 'angin', 'cuaca_ekstrem', 'hujan'):
            deflag = False
    for token in df_dominant_topic['Title'].iloc[i]:
        if token in ('kebakaran', 'banjir', 'gempa', 'gempa_bumi', 'longsor', 'banjir_rob', 'angin_kencang', 'angin', 'cuaca_ekstrem', 'hujan'):
            deflag = False
    if flag != deflag:
        total += 1
        idxs.append(i)
    elif flag == deflag and flag == False:
        total_tp += 1
        idxs_tp.append(i)
    elif flag == deflag and flag == True:
        total_tn += 1
        idxs_tn.append(i)
print(total)
print(total_tp)
print(total_tn)

In [ ]:
print(idxs_tn)

In [ ]:
# text  = preprocessing(old_ekonomi['desc'][old_ekonomi['desc'].str.find('hari') != -1].iloc[0])
# text
i = idxs_tn[0]
print(df_dominant_topic['Keywords'].iloc[i])
print(type(df_dominant_topic['Keywords'].iloc[i]))
print(df_dominant_topic['Text'].iloc[i])
print(type(df_dominant_topic['Text'].iloc[i]))
print(df_dominant_topic['Keywords'].iloc[i].split(", "))
print(lda_what_ekonomi[i])


In [ ]:
before_what_ekonomi = old[old['berita_topik_id'] == 1]
before_what_ekonomi = before_what_ekonomi[['berita_title', 'qe_what', 'berita_desc']]
before_what_ekonomi = before_what_ekonomi.dropna(subset=['berita_desc'])
before_what_ekonomi.sort_values('berita_desc', inplace=True)
before_what_ekonomi.drop_duplicates(subset='berita_desc', keep=False, inplace=True)
# print(before_what_ekonomi)
# print(before_what_ekonomi['qe_what'].iloc[i])
# print(before_what_ekonomi['berita_title'].iloc[i])
print(before_what_ekonomi['berita_desc'].iloc[i])

In [ ]:
total_empty = 0
empty_val = []
with open(PATH + ekonomi + 'ture_negative_4.txt', 'w', encoding='UTF8') as f:
    nl = "\n"
    dashes = '--------------------------------------'
    for x in idxs_tn:
        f.write(dashes + nl)
        f.write(before_what_ekonomi['berita_title'].iloc[x] + nl)
        f.write(before_what_ekonomi['qe_what'].iloc[x] + nl)
        f.write(str(lda_what_ekonomi[x]) + nl)
        # print(before_what_ekonomi['berita_title'].iloc[x])
        # print(before_what_ekonomi['qe_what'].iloc[x])
        # print(lda_what_ekonomi[x])

        if len(lda_what_ekonomi[x])<3:
            total_empty += 1
            empty_val.append(x)

print(total_empty)
print(empty_val)


In [ ]:
hitung = Counter()
for i in idxs_tn:
    # hitung.update(lda_what_ekonomi[i])
    hitung.update(lda_title_what[i])

In [ ]:
hitung